# In Depth Tutorial on the MyUR3e Class #

## Introduction

The MyUR3e class is a part of the myur library and enables advanced control of the MyUR3e arm through simple Python commands. This tutorial provides a comprehensive guide to using the `MyUR3e` class for controlling the UR3e robot arm. The `MyUR3e` class offers various functionalities, including basic operations like reading joint states, controlling the gripper, and of course moving the arm. To see examples of how a full robot program is built using this class, check out other files in the example folder. 

We will walk through each feature with practical examples. It can be assumed that all units default to meters, degrees, newtons, and newtown meters unless otherwise noted.


## Initialization ##

In [ ]:
# Import the MyUR3e class from the myur library
from myur import MyUR3e

# Create an instance of the MyUR3e class
robot = MyUR3e()

## Reading Data Fields ##
The MyUR3e class has data fields for joint angles, velocities, efforts, end effector position, force, torque, and gripper settings. The `read` functions give access to the most recent data point for the desired value.

In [ ]:
import time
from IPython.display import clear_output

while True:
    clear_output(wait=True)

    # Joint Position: [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3] degrees
    print(f"Joint Position: {robot.read_joints_pos()}")

    # Joint Velocity: [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3] degrees / s
    print(f"Joint Velocity: {robot.read_joints_vel()}")

    # Joint Effort: [Pan, Lift, Elbow, Wrist1, Wrist2, Wrist3] amps?
    print(f"Joint Effort: {robot.read_joints_eff()}")

    # Global End Effector Position: [x,y,z,rx,ry,rz] meters & degrees
    print(f"Global End Effector Position: {robot.read_global_pos()}")

    # End Effector Force [x,y,z] # newtons (relative to end effector)
    print(f"End Effector Force: {robot.read_force()}")

    # End Effector Torque [rx,ry,rz] # newton meters (relative to end effector)
    print(f"End Effector Torque: {robot.read_torque()}")
    
    # Gripper Settings: [Position, Speed, Force] unitless scale (0-100)
    print(f"Gripper Settings: {robot.read_gripper()}")

    time.sleep(0.05)

## Movement
### Global & Joint Movement
You can move the robot arm using either global cartesian coordinates or joint angles. Both systems have advantages and disadvantages. Specifying global coordinates is certainly easier to use and will require less work. However, by only specifying the position of the end effector you are leaving the specific pose of the arm up to the IK solver (inverse kinematic solver). By default, the IK solver will find the pose closest to the arms current position. There are instances when this might not give you the desired pose and at that point you should specify the position using `move_joints`. Additionaly, using `move_joints` allows you to create isolated movements like panning the robot left or right or rotating the wrist.

<div style="display: flex; align-items: center;">
    <img src="img/ur3e_diagram.png" alt="Image 1" style="width:300px;"/>
</div>

In [ ]:
# Create a point using global coordinates
global_point = [0.2, 0.3, 0.3, 0.0, 0.0, 0.0] # [x,y,z,rx,ry,rz] meters, degrees

# Move the robot to a new pose using global end effector position
robot.move_global(global_point)

In [ ]:
# Create a point using joint positions that is 10cm away from global_point
# This specific point was found by moving globally and then reading the joint positions
joints_point = [17.45, -101.18, 101.04, -89.92, -90.13, -72.65]  # [pan,lift,elbow,wrist1,wrist2,wrist3] degrees

# Move the robot to a new pose using joint angles
robot.move_joints(joints_point)

If you are using both `move_global` and `move_joints`, be careful when switching between them. Joints can move more than 360 degrees, and as a result specified joint angles might not be the closest rotation to the robots current pose. It is also possible to pass joint angles into `move_global` and vice versa. More often than not this will produce an IK error but it is possible a solution will exist and the arm will move to an undesired pose.

If you would like to translate between coordinate systems, you can use the built in `solve_ik` and `solve_fk` to do so. Keep in mind that the inverse kinematics will return the joint angles closest to the robot's current pose unless you specify another reference pose using the `q_guess` parameter in radians.

In [ ]:
# Get joint angles from a global point using inverse kinematics
print("Joint Angle: ", robot.solve_ik(global_point))

# Get global point from joint angles using forward kinematics
print("Global: ", robot.solve_fk(joints_point))

### Relative & Absolute Movement
When using `move_global` or `move_joints` you are specifying the absolute location you want the end effector or joints to move to. If you don't know or care exactly where the arm is however, it might useful to move the arm using relative commands. For example, if you were receiving directions to navigate a city an absolute command might be "walk to 33rd and Broadway" whereas a relative command would be "walk 4 blocks North and 2 blocks East." You can make relative movement commands using `move_global_r` and `move_joints_r`. With relative commands, a zero represents no movement in that axis. 

In [ ]:
# Move 10cm in the x axis using relative motion
relative_global = [[0.1, 0.0, 0.0, 0.0, 0.0, 0.0]]  # [x,y,z,rx,ry,rz] meters, degrees

# Move the robot using the relative command
# QUESTION: explore time with relative trajectories
robot.move_global_r(relative_global)

In [ ]:
# Move the pan joint 45 degrees using relative motion
relative_joints = [[45, 0.0, 0.0, 0.0, 0.0, 0.0]]  # [pan,lift,elbow,wrist1,wrist2,wrist3] degrees

# Move the robot using the relative command
robot.move_joints_r(relative_joints)

### The Gripper
The UR3e has a gripper attached to the end effector that can be used for picking up and grasping objects. You can open and close it using the `move_gripper` function.

In [ ]:
# Open the gripper
robot.move_gripper(100) # 100 = closed
robot.move_gripper(0) # 0 = open

For more fine control over the gripper, you can pass through speed and force parameters. The default for speed and force is 50. Try putting different objects in the gripper and playing with speed and force to see the results! (Do not put your hand or other parts of your body in the gripper, it is strong!) 

In [ ]:
# Close the gripper
robot.move_gripper(100, 50, 50)  # position=100, speed=50, force=50
robot.read_gripper()  # Read the gripper to determine its final position

### Trajectories
If you want the arm to move in a specific motion, you can do so with trajectories! Instead of passing through a single point, we will just pass through a list of many points that the arm will move to in a consecutive manner.

In [ ]:
# Create a two point trajectory using global coordinates
trajectory = [[0.2, 0.3, 0.3, 0.0, 0.0, 0.0], 
              [0.2, 0.2, 0.3, 0.0, 0.0, 0.0]]

# Move the end effector through the trajectory
robot.move_global(trajectory)

You can do the same thing with relative commands. Each "point" in a relative trajectory is based on the point before it.

In [ ]:
# Move in a 10cm square using global relative commands
relative_trajectory = [
    [0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.1, 0.0, 0.0, 0.0, 0.0],
    [-0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, -0.1, 0.0, 0.0, 0.0, 0.0],
]

# Move the end effector through the trajectory
robot.move_global_r(relative_trajectory)

Trajectories can also be made with joint positions.

### Time
The default time of a given trajectory is to take 5 seconds. Additionally, out of precaution the arm will take 5 seconds to move to the first pose of a given trajectory. The `time` parameter allows you to alter both of these conditions, however it is important that you do this very intentionally as an incorrectly specified time parameter can result in dangerously fast arm movement.

In [ ]:
# Take the default 5 seconds to arrive at the trajectory and then 3 to complete it
robot.move_global(trajectory, time=3)

In [ ]:
# Take 4 seconds to arrive at the trajectory and then 3 to complete it
robot.move_global(trajectory, time=(4, 3))

In [ ]:
# Move the end effector at a constant velocity of 0.3m/s to the trajectory and 0.4m/s during the trajectory
robot.move_global(trajectory, time=('cv',0.3,0.4))

Relative movements are slightly different because there is no first pose of the trajectory. Instead the first pose is wherever the robot currently is. For that reason the time parameter of a relative movement cannot be specified as a tuple, and the parameter represents the total trajectory time.

In [ ]:
# Take 5 seconds to complete the trajectory
robot.move_global_r(relative_trajectory, time=5)

### Interpolation
If you look very closely, you might notice that the arm's end effector doesn't always follow a straight line between points. This is because the arm found the most efficient path from A to B, which doesn't always result in linear movement. Lets create a new trajectory to exagerate this. Watch the path of the end effector as the arm moves.

In [ ]:
trajectory = [[0.2, 0.2, 0.3, 0.0, 0.0, 0.0], [0.2, -0.2, 0.3, 0.0, 0.0, 0.0]]

robot.move_global(trajectory, time=5)

 Did you see the curve? We can fix this with the `interp` parameter. This will fill in our trajectory with lots of points in between the ones that we have specified to ensure that the end effector follows the desired path.

In [ ]:
# Interpolate the trajectory between given points
robot.move_global(trajectory,time=5,interp="linear")

In addition to linear interpolation, there are also options for arc and spline interpolations (think first order, second order, third order polynomials). As a result the linear interpolation requires a minimum trajectory of two points, the arc requires three, and the spline requires four. Lets create a more interesting trajectory to try this out!

In [ ]:
zigzag = [
    [0.3, 0.3, 0.3, 0, 0, 0],
    [0.2, 0.2, 0.3, 0, 0, 0],
    [0.4, 0.1, 0.3, 0, 0, 0],
    [0.3, 0.0, 0.3, 0, 0, 0],
]

# First run the zigzag with no interpolation
# Watch the end effector arc between points
robot.move_global(zigzag, time=(5, 10))

In [ ]:
# Interpolate the points with a first order equation
robot.move_global(zigzag, time=(5, 10), interp="linear")

In [ ]:
# Interpolate the points with a second order equation
robot.move_global(zigzag, time=(5, 10), interp="arc")

In [ ]:
# Interpolate the points with a third order equation
robot.move_global(zigzag, time=(5, 10), interp="spline")

## Trajectory Features
### Trajectory Visualization
The difference between each interpolation method might be hard to see as the robot moves. Lets try visualizing the trajectories instead! Whenever you run a trajectory using a move command the points will automatically be added to a 3D plot stored in a local html file. You can open the HTML file in your Jupyter Notebook with the code below or by opening the file in your browser. Up to this point all trajectories will have been added to this plot so it might be messy!

In [ ]:
from IPython.display import HTML

# Open HTML file as a cell output in your Jupyter Notebook
HTML(filename="ur3e_trajectory.html")

When debugging your program you might want to add a trajectory to the visualization without moving the robot, or clear all trajectories from the plot to start over. You can do so with the `clear_vis` function and the `vis_only` parameter.

In [ ]:
# Clear all trajectories from the plot
robot.clear_vis()

# Add a trajectory to the plot without moving the robot
robot.move_global(zigzag, interp="linear", vis_only=True)
robot.move_global(zigzag, interp="arc", vis_only=True)
robot.move_global(zigzag, interp="spline", vis_only=True)

HTML(filename="ur3e_trajectory.html")

When viewing a plot, colored points represent end effector locations and grey segments represent the orientation of the end effector. The base of the robot is at the origin of the plot. To hide a trajectory or orientation click on the label in the legend. To isolate a trajectory or orientation double click the label in the legend. To get more information about a trajectory point, hover your mouse over the point. If your trajectories are returning IK errors, the unsolved points will appear as larger red markers in the plot.

### Saving Trajectories
When programming the robot, you might create a trajectory that you want to save in your code without defining the lengthy coordinates in visible text. The `save_trajectory` and `get_trajectory` functions will allow you to label, store, and access trajectories in a local .json file.

In [ ]:
# Save the trajectory zigzag with the name "interpolation_test"
robot.save_trajectory("interpolation_test",zigzag)

# Access the saved trajectory named "interpolation_test"
interpolation_test = robot.get_trajectory("interpolation_test")

You can also run your saved trajectories directly inside of a move function.

In [ ]:
# Move the robot along the interpolation test trajectory using spline interpolation
robot.move_global("interpolation_test", time=(3, 10), interp="spline")

### Recording Trajectories
Some trajectories are harder to parametrically define than others, and if you don't care much about precision then you might want to just move the robot yourself and ask it to mirror you later! This is also useful when it is important *how* the robot moves from point A to point B. If you only specify A and B, the robot might not do what you want in between. If you use the `record` function however you can take complete control of the robot's motion. This is a great solution to transitional trajectories that are moving your end effector from one active location to another.

In [ ]:
# Record a trajectory called my_motion
robot.record("my_motion")

To use the record function, pause the UR3e's external control program using the Teach Pendant. Run the record function and put the UR3e into FreeDrive. The recording will start as soon as the arm moves and end once it comes to a stop. The default function will record a point every second. You can vary this using the `interval` parameter as well as change the threshold for movement detection. A large interval will result in smoother motion when paired with spline interpolation. A smaller interval will give you more fine control over the robots movement but likely appear more jerky.

In [ ]:
# This will record fewer points (every 2 seconds)
robot.record("my_motion2",interval=2,threshold=0.001)

Play back your recordings using the `move_joints()` function. Don't forget to press resume on the Teach Pendant!

In [ ]:
robot.move_joints("my_motion",time=(5,10),interp="spline")

In [ ]:
robot.move_joints("my_motion2", time=(5, 10), interp="spline")

## ROS Logs
You may have noticed the red highlighted outputs that show up whenever you run a move command. These are logs from the ROS2 server that give more information about the processes that are being executed. If you are having issues with the ROS2 server, it might be helpful to see more detailed logs. You can enable DEBUG logs using the `set_debug_level(True)` function.

In [ ]:
# Turn on debug logs
robot.set_debug_level(True)

# Turn off debug logs
robot.set_debug_level(False)

You can also add your own messages to the log using the `print_info` and `print_debug` functions. This is a preffered method of printing output over the standard `print()` function if you are sending lots of move commands to the robot or running non blocking code. The one disadvantage of this method is that you have less control of the formatting of your print outs.

In [ ]:
robot.print_info("Relevant information to my code")

robot.print_debug("More detailed relevant information to my code")

## Non Blocking Commands
All move commands (move_global, move_global_r, move_joints, move_joints_r, move_gripper) take time to complete. In the context of the code, this means that we don't move on from the move command until it is finished. But what if we don't want to wait around for that to happen? All of the move commands have a `wait` parameter that allows us to do just that. If we set `wait=False` we can move on to other code while the move command is executing. For example, we might want to monitor the force sensor, close the gripper, or print out values while the arm is moving to a new pose.

In [ ]:
import time
import matplotlib.pyplot as plt

# Move the arm along the interpolation_test trajectory using wait=False
robot.move_global("interpolation_test", time=(5, 5), interp="spline", wait=False)

# While the arm is moving, measure the force in the x axis at the end effector.
data = [[], [], []]
axis_dict = {
    0: "X Axis",
    1: "Y Axis",
    2: "Z Axis",
}
time.sleep(4)
for i in range(600):
    clear_output(wait=True)
    force = robot.read_force()
    for i, axis in enumerate(data):
        axis.append(robot.read_force()[i])
    print(force)
    time.sleep(0.01)

# Plot the force data
for i, axis in enumerate(data):
    plt.plot(axis, linestyle="-", label=axis_dict[i])
plt.legend()
plt.show()

If the code worked, you should see a plot of forces in the x, y, and z directions while the arm was moving. This also goes to show that the force sensor is not immune to the acceleration of the arm!

You might also want to stop the arm while it is moving if some condition is met. For example, if the force sensor goes above a certain threshold. This can be done using the `stop` command. 

In [ ]:
# Move the robot along a five second trajectory with a non blocking command 
robot.move_global(trajectory,time=5,wait=False)

# Wait six seconds (should finish 1 second into the trajectory)
time.sleep(6)

# Stop the robot at its current location
robot.stop()

## Callbacks
When running non blocking code, you might want to know more about where you are in the process of communicating with the robot. The set callback functions enable a level of customization for this purpose. By setting each callback to a function that you define in your own code, you can trigger custom logic upon events in the communication sequence. 

For example, a function set with the `set_sent_callback` will be called when a trajectory is successfully sent to the robot. A function set with the `set_finished_callback` will be called when a trajectory is successfully finished by the robot. Finally, a function set with the `set_timer_callback` will be called at a regular period defined by your code.

In [ ]:
def my_function1():
    robot.print_info("My trajectory was sent")


# Set the sent callback to my_function1
robot.set_sent_callback(my_function1)


def my_function2():
    robot.print_info("My trajectory was finished")


# Set the finished callback to my_function2
robot.set_finished_callback(my_function2)


def my_function3():
    robot.print_info("My timer went off")


# Set the timer callback to my_function3
robot.set_timer_callback(my_function3, period=2)

# Move the robot over a 15 seconds
robot.move_global(trajectory, time=(5, 10))

It is important to note what is considered good practice for these callbacks. When a callback is raised, it is added to a queue of callbacks that are waiting to be executed. These callbacks are crucial for the robot to run in a timely manner and are the foundation of ROS2. Therefore, your callbacks should be short and concise so that they do not bog down the queue.